In [1]:
#menyambungkan file ke google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install Sastrawi

     |████████████████████████████████| 209 kB 7.3 MB/s 


In [3]:
import pandas as pd
import numpy as np
import re
import string
import nltk
nltk.download('punkt')
import nltk
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from wordcloud import WordCloud

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
scrapeddata = pd.read_csv("/content/drive/My Drive/TA/hasilpreprocessing.csv")
scrapeddata

,review,rating,versi,date
0,"Setelah update, sertifikat malah hilang padaha...",5,4.1.0,2022-02-15 15:09:48
1,saya sudah divaksin 3 kali alhamdulillah sehat...,5,4.0.8,2022-02-14 14:24:24
2,masalah mengatur tanggal nya bikin kesel😡,1,4.1.0,2022-02-14 14:05:08
3,Cukup pake no nik saja apakah sudah vaksin tid...,4,4.1.0,2022-02-14 14:05:01
4,Ngikut aja bos,5,4.1.0,2022-02-14 14:04:23
...,...,...,...,...
49995,Tolong dong klo bikin sertifikat vaksin .tangg...,5,3.4.4,2021-09-03 04:14:27
49996,"Jadi bingung sama aplikasi ini , saya sdh 2 ka...",2,3.4.4,2021-09-03 04:09:43
49997,Aplikasinya bagus...tapi sayang untuk scan bar...,3,3.4.4,2021-09-03 04:08:17
49998,"Sudah di vaksin, sudah masukin data, tapi GA a...",1,3.4.4,2021-09-03 04:05:18


In [5]:
#menghapus kolom yang tidak dibutuhkan
df = scrapeddata.drop(scrapeddata.columns[[2, 3]], axis=1)
df

,review,rating
0,"Setelah update, sertifikat malah hilang padaha...",5
1,saya sudah divaksin 3 kali alhamdulillah sehat...,5
2,masalah mengatur tanggal nya bikin kesel😡,1
3,Cukup pake no nik saja apakah sudah vaksin tid...,4
4,Ngikut aja bos,5
...,...,...
49995,Tolong dong klo bikin sertifikat vaksin .tangg...,5
49996,"Jadi bingung sama aplikasi ini , saya sdh 2 ka...",2
49997,Aplikasinya bagus...tapi sayang untuk scan bar...,3
49998,"Sudah di vaksin, sudah masukin data, tapi GA a...",1


In [6]:
def casefoldingText(review):
    review = re.sub(r'[0-9]+', '', review) # menghapus angka
    review = re.sub(r"http\S+", '', review) # menghapus link

    review = review.lower() # mengubah mengubah semua text menjadi huruf kecil
    review = review.translate(str.maketrans('', '', string.punctuation)) # menghapus tanda baca
    review = review.strip('') # menghapus spasi/karakter kosong
    return review

df['casefolding'] = df['review'].apply(lambda x: casefoldingText(x))
df

,review,rating,casefolding
0,"Setelah update, sertifikat malah hilang padaha...",5,setelah update sertifikat malah hilang padahal...
1,saya sudah divaksin 3 kali alhamdulillah sehat...,5,saya sudah divaksin kali alhamdulillah sehat ...
2,masalah mengatur tanggal nya bikin kesel😡,1,masalah mengatur tanggal nya bikin kesel😡
3,Cukup pake no nik saja apakah sudah vaksin tid...,4,cukup pake no nik saja apakah sudah vaksin tid...
4,Ngikut aja bos,5,ngikut aja bos
...,...,...,...
49995,Tolong dong klo bikin sertifikat vaksin .tangg...,5,tolong dong klo bikin sertifikat vaksin tangga...
49996,"Jadi bingung sama aplikasi ini , saya sdh 2 ka...",2,jadi bingung sama aplikasi ini saya sdh kalu...
49997,Aplikasinya bagus...tapi sayang untuk scan bar...,3,aplikasinya bagustapi sayang untuk scan barcod...
49998,"Sudah di vaksin, sudah masukin data, tapi GA a...",1,sudah di vaksin sudah masukin data tapi ga ada...


In [8]:
def tokenizingText(review): # proses memisahkan teks menjadi potongan token 
    review = word_tokenize(review) #memisahkan kalimat menjadi potongan kata yang disebut token
    return review
df['tokenizing'] = df['casefolding'].apply(lambda x: tokenizingText(x))
df

,review,rating,casefolding,tokenizing
0,"Setelah update, sertifikat malah hilang padaha...",5,setelah update sertifikat malah hilang padahal...,"[setelah, update, sertifikat, malah, hilang, p..."
1,saya sudah divaksin 3 kali alhamdulillah sehat...,5,saya sudah divaksin kali alhamdulillah sehat ...,"[saya, sudah, divaksin, kali, alhamdulillah, s..."
2,masalah mengatur tanggal nya bikin kesel😡,1,masalah mengatur tanggal nya bikin kesel😡,"[masalah, mengatur, tanggal, nya, bikin, kesel😡]"
3,Cukup pake no nik saja apakah sudah vaksin tid...,4,cukup pake no nik saja apakah sudah vaksin tid...,"[cukup, pake, no, nik, saja, apakah, sudah, va..."
4,Ngikut aja bos,5,ngikut aja bos,"[ngikut, aja, bos]"
...,...,...,...,...
49995,Tolong dong klo bikin sertifikat vaksin .tangg...,5,tolong dong klo bikin sertifikat vaksin tangga...,"[tolong, dong, klo, bikin, sertifikat, vaksin,..."
49996,"Jadi bingung sama aplikasi ini , saya sdh 2 ka...",2,jadi bingung sama aplikasi ini saya sdh kalu...,"[jadi, bingung, sama, aplikasi, ini, saya, sdh..."
49997,Aplikasinya bagus...tapi sayang untuk scan bar...,3,aplikasinya bagustapi sayang untuk scan barcod...,"[aplikasinya, bagustapi, sayang, untuk, scan, ..."
49998,"Sudah di vaksin, sudah masukin data, tapi GA a...",1,sudah di vaksin sudah masukin data tapi ga ada...,"[sudah, di, vaksin, sudah, masukin, data, tapi..."


In [9]:
def filteringText(review): # menghapus stopword/kata-kata yang memiliki arti rendah
    listStopwords = set(stopwords.words('indonesian'))
    filtered = []
    for rvw in review:
        if rvw not in listStopwords:
            filtered.append(rvw)
    review = filtered 
    return review

df['stopword_removal'] = df['tokenizing'].apply(lambda x: filteringText(x))
df

,review,rating,casefolding,tokenizing,stopword_removal
0,"Setelah update, sertifikat malah hilang padaha...",5,setelah update sertifikat malah hilang padahal...,"[setelah, update, sertifikat, malah, hilang, p...","[update, sertifikat, hilang, kemarin, update]"
1,saya sudah divaksin 3 kali alhamdulillah sehat...,5,saya sudah divaksin kali alhamdulillah sehat ...,"[saya, sudah, divaksin, kali, alhamdulillah, s...","[divaksin, kali, alhamdulillah, sehat, terima,..."
2,masalah mengatur tanggal nya bikin kesel😡,1,masalah mengatur tanggal nya bikin kesel😡,"[masalah, mengatur, tanggal, nya, bikin, kesel😡]","[mengatur, tanggal, nya, bikin, kesel😡]"
3,Cukup pake no nik saja apakah sudah vaksin tid...,4,cukup pake no nik saja apakah sudah vaksin tid...,"[cukup, pake, no, nik, saja, apakah, sudah, va...","[pake, no, nik, vaksin, tidaknya, kalo, tangga..."
4,Ngikut aja bos,5,ngikut aja bos,"[ngikut, aja, bos]","[ngikut, aja, bos]"
...,...,...,...,...,...
49995,Tolong dong klo bikin sertifikat vaksin .tangg...,5,tolong dong klo bikin sertifikat vaksin tangga...,"[tolong, dong, klo, bikin, sertifikat, vaksin,...","[tolong, klo, bikin, sertifikat, vaksin, tangg..."
49996,"Jadi bingung sama aplikasi ini , saya sdh 2 ka...",2,jadi bingung sama aplikasi ini saya sdh kalu...,"[jadi, bingung, sama, aplikasi, ini, saya, sdh...","[bingung, aplikasi, sdh, kalu, vaksin, tp, ser..."
49997,Aplikasinya bagus...tapi sayang untuk scan bar...,3,aplikasinya bagustapi sayang untuk scan barcod...,"[aplikasinya, bagustapi, sayang, untuk, scan, ...","[aplikasinya, bagustapi, sayang, scan, barcode..."
49998,"Sudah di vaksin, sudah masukin data, tapi GA a...",1,sudah di vaksin sudah masukin data tapi ga ada...,"[sudah, di, vaksin, sudah, masukin, data, tapi...","[vaksin, masukin, data, ga, sertifikat, nya]"
